In [ ]:
# ==========================================================
# Celda de Configuración Inicial para Google Colab
# ==========================================================
# Ejecuta esta celda al principio de tu notebook cada vez que inicies una nueva sesión.

# --- 1. Conectar a Google Drive ---
# Aparecerá una ventana para que autorices el acceso a tus archivos.
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Drive conectado exitosamente.")

import os
import time
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import ipywidgets as widgets
from IPython.display import display

Mounted at /content/drive

✅ Google Drive conectado exitosamente.


In [ ]:
# ==========================================================
# SECCIÓN DE CONFIGURACIÓN INTERACTIVA
# ==========================================================

# Opción 1: Menú Desplegable (RECOMENDADO)
model_options = ['Seleccione un modelo', 'mathbert', 'word2vec']
model_selector = widgets.Dropdown(
    options=model_options,
    value=model_options[0],
    description='Modelo:',
    disabled=False,
)

print("Por favor, selecciona un modelo para el cual calcular el Índice KI y luego ejecuta la celda de abajo.")
display(model_selector)

Por favor, selecciona un modelo para el cual calcular el Índice KI y luego ejecuta la celda de abajo.


Dropdown(description='Modelo:', options=('Seleccione un modelo', 'mathbert', 'word2vec'), value='Seleccione un…

In [ ]:
# ==========================================================
# 1. CONFIGURACIÓN
# ==========================================================
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/TFM_Cienciometria'
MODEL_NAME = model_selector.value

# --- PARÁMETROS DEL ÍNDICE KI ---
# [cite_start]Ventana de 5 años hacia atrás y 5 hacia adelante, como en el paper [cite: 243]
YEARS_WINDOW = 5

# --- PARÁMETRO DE OPTIMIZACIÓN ---
# 8192 o 16384 son buenos valores para una A100.
BATCH_SIZE = 16384

print(f"✅ Modelo seleccionado: {MODEL_NAME}")
print(f"✅ Ventana de cálculo para KI: {YEARS_WINDOW} años (pasado y futuro)")

# Configurar rutas y directorios basados en la selección
if MODEL_NAME == 'mathbert':
    TMP_DIR = os.path.join(DRIVE_PROJECT_PATH, 'data/mathbert/tmp')
elif MODEL_NAME == 'word2vec':
    TMP_DIR = os.path.join(DRIVE_PROJECT_PATH, 'data/word2vec/tmp')
else:
    # Este bloque se ejecuta si no se ha seleccionado un modelo válido
    pass

# ==========================================================
# 2. LÓGICA DE CÁLCULO DEL ÍNDICE KI SEMÁNTICO
# ==========================================================
def compute_ki_semantic(df: pd.DataFrame, years_window: int, batch_size: int):
    """
    Calcula el Índice KI Semántico usando una ventana temporal simétrica (pasado/futuro)
    y vectores de embeddings pre-calculados. El cálculo se optimiza para GPU.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🚀 Realizando cálculos del Índice KI en {device} con precisión FP16 y BATCH_SIZE={batch_size}")

    # --- Pre-cálculos de fechas y ORDENAMIENTO CRÍTICO ---
    df['cover_date'] = pd.to_datetime(df['cover_date'], errors="coerce")
    df = df.dropna(subset=['cover_date', 'vector']).sort_values(by=["cover_date", "eid"]).reset_index(drop=True)
    print("✅ DataFrame ordenado por 'cover_date' y 'eid'.")

    # Convertir fechas a números para búsquedas rápidas con NumPy
    dates_ordinal = df['cover_date'].apply(lambda x: x.toordinal()).to_numpy()

    # Determinar el rango de fechas procesable para tener ventanas completas
    time_delta = pd.Timedelta(days=365.25 * years_window)
    min_date = df['cover_date'].min() + time_delta
    max_date = df['cover_date'].max() - time_delta

    # Filtrar para procesar solo los artículos que tienen datos suficientes
    processing_mask = (df['cover_date'] >= min_date) & (df['cover_date'] <= max_date)
    processing_indices = df.index[processing_mask].to_numpy()

    print(f"ℹ️  Corpus total: {len(df)} artículos.")
    print(f"✅ Se procesarán {len(processing_indices)} artículos entre {min_date.date()} y {max_date.date()} para garantizar ventanas completas.")

    # --- Carga y normalización de vectores en la GPU ---
    V = np.vstack(df["vector"].values).astype(np.float32)
    V_gpu = torch.from_numpy(V).to(device, dtype=torch.float16)
    V_gpu = V_gpu / torch.norm(V_gpu, dim=1, keepdim=True).clamp(min=1e-12)

    results = []
    desc = f"🔍 Calculando KI Semántico (Ventana={years_window} años)"

    # Bucle principal por lotes (solo sobre los índices procesables)
    for i in tqdm(range(0, len(processing_indices), batch_size), desc=desc):
        batch_indices = processing_indices[i : i + batch_size]

        for j in batch_indices:
            target_vector = V_gpu[j]
            current_date_ord = dates_ordinal[j]
            window_days = time_delta.days

            # --- Ventana PASADA ---
            past_start_ord = current_date_ord - window_days
            start_idx = np.searchsorted(dates_ordinal, past_start_ord, side='left')
            past_indices = np.arange(start_idx, j)

            if len(past_indices) > 0:
                past_sims = (V_gpu[past_indices] @ target_vector + 1) / 2
                sum_past_sim = torch.sum(past_sims, dtype=torch.float32).clamp(min=1e-9) # Clamp para evitar división por cero
            else:
                sum_past_sim = torch.tensor(1e-9, device=device)

            # --- Ventana FUTURA ---
            future_end_ord = current_date_ord + window_days
            end_idx = np.searchsorted(dates_ordinal, future_end_ord, side='right')
            future_indices = np.arange(j + 1, end_idx)

            if len(future_indices) > 0:
                future_sims = (V_gpu[future_indices] @ target_vector + 1) / 2
                sum_future_sim = torch.sum(future_sims, dtype=torch.float32).clamp(min=1e-9)
            else:
                sum_future_sim = torch.tensor(1e-9, device=device)

            # --- Cálculo del Índice KI Semántico ---
            ki_score = torch.log(sum_future_sim / sum_past_sim)

            results.append({
                "eid": df.at[j, "eid"],
                "ki_score": ki_score.item(),
                "past_pool_size": len(past_indices),
                "future_pool_size": len(future_indices)
            })

    return pd.DataFrame(results)

# ==========================================================
# 3. EJECUCIÓN PRINCIPAL
# ==========================================================
if __name__ == '__main__':
    if MODEL_NAME != 'Seleccione un modelo':
        t0 = time.time()
        BASE = "TODO"

        # Generar nombres de archivo
        input_path = os.path.join(TMP_DIR, f"04_vectores_{MODEL_NAME}_{BASE}.pkl")
        output_suffix = f"05_ki_semantic_{MODEL_NAME}_{BASE}_win{YEARS_WINDOW}"
        csv_output = os.path.join(TMP_DIR, f"{output_suffix}.csv")
        pkl_output = os.path.join(TMP_DIR, f"{output_suffix}.pkl")

        print("="*50)
        print(f"INICIANDO CÁLCULO PARA EL MODELO: {MODEL_NAME}")
        print(f"CARGANDO VECTORES DESDE: {input_path}")
        print("="*50)
        df_main = pd.read_pickle(input_path)

        # --- LLAMADA A LA FUNCIÓN DE CÁLCULO ---
        ki_df = compute_ki_semantic(
            df=df_main,
            years_window=YEARS_WINDOW,
            batch_size=BATCH_SIZE
        )

        print("\n📊 Resumen del Índice KI Semántico:")
        if not ki_df.empty:
            print(ki_df['ki_score'].describe())
        else:
            print("No se generaron resultados. Revisa el rango de fechas de tu corpus.")

        print("\n💾 Guardando resultados...")
        meta_cols = [c for c in ["eid", "title", "cover_date", "citedby_count", "doi"] if c in df_main.columns]
        # Usamos 'inner' join para quedarnos solo con los artículos para los que se pudo calcular el KI
        final_df = df_main[meta_cols].merge(ki_df, on="eid", how="inner")

        final_df.to_csv(csv_output, index=False)
        final_df.to_pickle(pkl_output)

        elapsed = time.time() - t0
        print(f"\n✅ Resultados guardados en:\n- {csv_output}\n- {pkl_output}")
        print(f"⏱️  Tiempo total: {elapsed/60:.2f} min")
    else:
        print("⚠️ Por favor, selecciona un modelo en el menú desplegable y vuelve a ejecutar la celda.")

✅ Modelo seleccionado: word2vec
✅ Ventana de cálculo para KI: 5 años (pasado y futuro)
INICIANDO CÁLCULO PARA EL MODELO: word2vec
CARGANDO VECTORES DESDE: /content/drive/MyDrive/TFM_Cienciometria/data/word2vec/tmp/04_vectores_word2vec_TODO.pkl
